In [6]:
import pandas as pd
import os

In [81]:
def findFileByName(folder, prefix):
    return [filename for filename in os.listdir(folder) if filename.startswith(prefix)]

# folder: "data", "metadata", or "titles"
def searchAndLoadData(folder, prefix):
    # Extract the first result
    results = findFileByName(folder, prefix)
    file = results[0]
    filepath = os.path.join(folder, file)
    
    # Print result
    #print("Found", file, "with", data.shape[0], "rows")
    if len(results) > 1:
        pass#print("Also found:", results[1:])
    
    # Load data
    df = pd.read_csv(filepath)
    #print("Loaded", df.shape[0], "rows, columns=", df.columns.values)
    #print()
    return df

data.head()
#metadata.head()

,id,census_tract,county,estimate,error
1,1400000US06001400100,4001,Alameda County,2701,206
2,1400000US06001400200,4002,Alameda County,2050,140
3,1400000US06001400300,4003,Alameda County,4719,327
4,1400000US06001400400,4004,Alameda County,3930,450
5,1400000US06001400500,4005,Alameda County,3506,251


In [94]:
years = ["2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019"]

def getDataForYear(year):
    # The start of the file name to search for. Change the year to find other years' datasets.
    prefix = "ACSDT5Y" + year

    data = searchAndLoadData("data", prefix)
    metadata = searchAndLoadData("metadata", prefix)

    data = data.rename(columns=data.iloc[0]).drop(data.index[0])
    data[["census_tract","county","state"]]=data["Geographic Area Name"].str.split(",",expand=True)
    data["census_tract"] = data["census_tract"].str.replace(r'[a-zA-Z%]', '', regex=True)
    data["estimate"] = data["Estimate!!Total"]
    data["error"] = data["Margin of Error!!Total"]
    data = data.assign(year=year)

    # Reorganize
    data = data[["id", "census_tract", "county", "estimate", "error", "year"]]
    return data

result = pd.DataFrame(columns = ["id", "census_tract", "county", "estimate", "error", "year"]) 
for year in years:
    dataForYear = getDataForYear(year)
    result = pd.concat([result, dataForYear])
result = result.reset_index(drop=True)

print("Total", result.shape[0], "rows, columns=", result.columns.values)
print(result)

result.head()

Total 80570 rows, columns= ['id' 'census_tract' 'county' 'estimate' 'error' 'year']
                         id census_tract           county estimate error  year
0      1400000US06001400100         4001   Alameda County     2701   206  2010
1      1400000US06001400200         4002   Alameda County     2050   140  2010
2      1400000US06001400300         4003   Alameda County     4719   327  2010
3      1400000US06001400400         4004   Alameda County     3930   450  2010
4      1400000US06001400500         4005   Alameda County     3506   251  2010
...                     ...          ...              ...      ...   ...   ...
80565  1400000US06115040800          408      Yuba County     4652   247  2019
80566  1400000US06115040901       409.01      Yuba County     2720   298  2019
80567  1400000US06115040902       409.02      Yuba County     1653   334  2019
80568  1400000US06115041000          410      Yuba County     7298   789  2019
80569  1400000US06115041100          411      Y

,id,census_tract,county,estimate,error,year
0,1400000US06001400100,4001,Alameda County,2701,206,2010
1,1400000US06001400200,4002,Alameda County,2050,140,2010
2,1400000US06001400300,4003,Alameda County,4719,327,2010
3,1400000US06001400400,4004,Alameda County,3930,450,2010
4,1400000US06001400500,4005,Alameda County,3506,251,2010


In [102]:
result.to_csv("tabular_data.csv", encoding='utf-8', index=False)

In [103]:
df = pd.read_csv("tabular_data.csv")
df

,id,census_tract,county,estimate,error,year
0,1400000US06001400100,4001.00,Alameda County,2701,206,2010
1,1400000US06001400200,4002.00,Alameda County,2050,140,2010
2,1400000US06001400300,4003.00,Alameda County,4719,327,2010
3,1400000US06001400400,4004.00,Alameda County,3930,450,2010
4,1400000US06001400500,4005.00,Alameda County,3506,251,2010
...,...,...,...,...,...,...
80565,1400000US06115040800,408.00,Yuba County,4652,247,2019
80566,1400000US06115040901,409.01,Yuba County,2720,298,2019
80567,1400000US06115040902,409.02,Yuba County,1653,334,2019
80568,1400000US06115041000,410.00,Yuba County,7298,789,2019
